In [1]:
from binn import Network, BINN
import pandas as pd

input_data = pd.read_csv("../data/test_qm.csv")
translation = pd.read_csv("../data/translation.tsv", sep="\t")
pathways = pd.read_csv("../data/pathways.tsv", sep="\t")

network = Network(
    input_data=input_data,
    pathways=pathways,
    mapping=translation,
)

binn = BINN(
    network=network,
    n_layers=4,
    dropout=0.2,
    validate=False,
    residual=True
)

/home/erikh/BINN/BINN/test-venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from util_for_examples import fit_data_matrix_to_network_input, generate_data
import torch
from pytorch_lightning import Trainer

design_matrix = pd.read_csv('../data/design_matrix.tsv', sep="\t")

protein_matrix = fit_data_matrix_to_network_input(
    input_data, features=network.inputs)

X, y = generate_data(protein_matrix, design_matrix=design_matrix)

dataloader = torch.utils.data.DataLoader(dataset=torch.utils.data.TensorDataset(torch.Tensor(X), torch.LongTensor(y)),
                                            batch_size=8,
                                            num_workers=12,
                                            shuffle=True)
trainer = Trainer(max_epochs=3)
trainer.fit(binn, dataloader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name     | Type             | Params
----------------------------------------------
0 | layers   | Sequential       | 365 K 
1 | loss     | CrossEntropyLoss | 0     
2 | loss_val | CrossEntropyLoss | 0     
----------------------------------------------
365 K     Trainable params
0         Non-trainable params
365 K     Total params
1.464     Total estimated model params size (MB)
The number of training batches (25) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2: 100%|██████████| 25/25 [00:00<00:00, 34.54it/s, loss=0.664, v_num=402, train_loss=0.668, train_acc=0.650]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 25/25 [00:00<00:00, 30.63it/s, loss=0.664, v_num=402, train_loss=0.668, train_acc=0.650]


In [3]:
from binn import BINNExplainer

explainer = BINNExplainer(binn)

In [4]:
test_data = torch.Tensor(X[5:10])
background_data = torch.Tensor(X[0:5])

importance_df = explainer.explain(test_data, background_data)
importance_df.head()

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


,source,target,source name,target name,value,type,source layer,target layer
0,1,497,A0M8Q6,R-HSA-166663,0.0,0,0,1
1,1,497,A0M8Q6,R-HSA-166663,0.0,1,0,1
2,1,954,A0M8Q6,R-HSA-198933,0.0,0,0,1
3,1,954,A0M8Q6,R-HSA-198933,0.0,1,0,1
4,1,539,A0M8Q6,R-HSA-2029481,0.0,0,0,1


In [5]:
from binn import ImportanceNetwork

IG = ImportanceNetwork(importance_df)

In [11]:
IG.plot_complete_sankey(multiclass=False, savename='img/test.png', node_cmap='Reds', edge_cmap='Blues')

In [14]:
IG.get_node('P15291')

246

In [8]:
query_node = 1

IG.plot_subgraph_sankey(query_node, upstream=False, savename='img/test.png', cmap='BuGn')